# CSV Loader


Hướng dẫn này cung cấp một hướng dẫn toàn diện về cách sử dụng tiện ích `CSVLoader` trong LangChain để tích hợp dữ liệu từ các tệp CSV vào ứng dụng của bạn một cách liền mạch. `CSVLoader` là một công cụ mạnh mẽ để xử lý dữ liệu có cấu trúc, cho phép các nhà phát triển trích xuất, phân tích cú pháp và sử dụng thông tin từ các tệp CSV trong framework LangChain.

[Giá trị được phân tách bằng dấu phẩy (CSV)](https://en.wikipedia.org/wiki/Comma-separated_values) là một trong những định dạng phổ biến nhất để lưu trữ và trao đổi dữ liệu.

`CSVLoader` đơn giản hóa quá trình tải, phân tích cú pháp và trích xuất dữ liệu từ các tệp CSV, cho phép các nhà phát triển kết hợp thông tin này một cách liền mạch vào quy trình làm việc LangChain.


## How to load CSVs

Tệp giá trị được phân tách bằng dấu phẩy (CSV) là tệp văn bản được phân tách sử dụng dấu phẩy để phân tách các giá trị. LangChain có thể giúp bạn tải các tệp CSV một cách dễ dàng—chỉ cần import `CSVLoader` để bắt đầu.

Mỗi dòng của tệp là một bản ghi dữ liệu và mỗi bản ghi bao gồm một hoặc nhiều trường, được phân tách bằng dấu phẩy.

Chúng ta sử dụng một tệp CSV mẫu cho ví dụ.


In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# Create CSVLoader instance
loader = CSVLoader(file_path="data/titanic.csv")

# Load documents
docs = loader.load()

for record in docs[:2]:
    print(record)

page_content='PassengerId: 1
Survived: 0
Pclass: 3
Name: Braund, Mr. Owen Harris
Sex: male
Age: 22
SibSp: 1
Parch: 0
Ticket: A/5 21171
Fare: 7.25
Cabin: 
Embarked: S' metadata={'source': 'data/titanic.csv', 'row': 0}
page_content='PassengerId: 2
Survived: 1
Pclass: 1
Name: Cumings, Mrs. John Bradley (Florence Briggs Thayer)
Sex: female
Age: 38
SibSp: 1
Parch: 0
Ticket: PC 17599
Fare: 71.2833
Cabin: C85
Embarked: C' metadata={'source': 'data/titanic.csv', 'row': 1}


In [2]:
print(docs[1].page_content)

PassengerId: 2
Survived: 1
Pclass: 1
Name: Cumings, Mrs. John Bradley (Florence Briggs Thayer)
Sex: female
Age: 38
SibSp: 1
Parch: 0
Ticket: PC 17599
Fare: 71.2833
Cabin: C85
Embarked: C


## Customizing the CSV parsing and loading

`CSVLoader` chấp nhận đối số từ khóa `csv_args` hỗ trợ tùy chỉnh các tham số được truyền cho `csv.DictReader` của Python. Điều này cho phép bạn xử lý nhiều định dạng CSV khác nhau, chẳng hạn như dấu phân cách tùy chỉnh, ký tự trích dẫn hoặc xử lý dòng mới cụ thể.

Xem tài liệu [mô-đun csv](https://docs.python.org/3/library/csv.html) của Python để biết thêm thông tin về `csv_args` được hỗ trợ và cách điều chỉnh phân tích cú pháp cho nhu cầu cụ thể của bạn.


In [3]:
loader = CSVLoader(
    file_path="./data/titanic.csv",
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
        "fieldnames": [
            "Passenger ID",
            "Survival (1: Survived, 0: Died)",
            "Passenger Class",
            "Name",
            "Sex",
            "Age",
            "Number of Siblings/Spouses Aboard",
            "Number of Parents/Children Aboard",
            "Ticket Number",
            "Fare",
            "Cabin",
            "Port of Embarkation",
        ],
    },
)

docs = loader.load()

print(docs[1].page_content)

Passenger ID: 1
Survival (1: Survived, 0: Died): 0
Passenger Class: 3
Name: Braund, Mr. Owen Harris
Sex: male
Age: 22
Number of Siblings/Spouses Aboard: 1
Number of Parents/Children Aboard: 0
Ticket Number: A/5 21171
Fare: 7.25
Cabin: 
Port of Embarkation: S


## Specify a column to identify the document source

Bạn nên sử dụng đối số `source_column` để chỉ định nguồn của các tài liệu được tạo từ mỗi hàng. Nếu không, `file_path` sẽ được sử dụng làm nguồn cho tất cả các tài liệu được tạo từ tệp CSV.

Điều này đặc biệt hữu ích khi sử dụng các tài liệu được tải từ tệp CSV trong một chain được thiết kế để trả lời các câu hỏi dựa trên nguồn của chúng.


In [5]:
loader = CSVLoader(
    file_path="./data/titanic.csv",
    source_column="PassengerId",  # Specify the source column
)

docs = loader.load()  

print(docs[1])
print("-----------")
print(docs[1].metadata)

page_content='PassengerId: 2
Survived: 1
Pclass: 1
Name: Cumings, Mrs. John Bradley (Florence Briggs Thayer)
Sex: female
Age: 38
SibSp: 1
Parch: 0
Ticket: PC 17599
Fare: 71.2833
Cabin: C85
Embarked: C' metadata={'source': '2', 'row': 1}
-----------
{'source': '2', 'row': 1}


## Generating XML document format

Ví dụ này trình bày cách tạo định dạng Tài liệu XML từ `CSVLoader`. Bằng cách xử lý dữ liệu từ tệp CSV, bạn có thể chuyển đổi các hàng và cột của nó thành biểu diễn XML có cấu trúc.


In [6]:
row = docs[1].page_content.split("\n")  # split by new line
row_str = "<row>"
for element in row:
    splitted_element = element.split(":")  # split by ":"
    value = splitted_element[-1]  # get value
    col = ":".join(splitted_element[:-1])  # get column name

    row_str += f"<{col}>{value.strip()}</{col}>"
row_str += "</row>"
print(row_str)

<row><PassengerId>2</PassengerId><Survived>1</Survived><Pclass>1</Pclass><Name>Cumings, Mrs. John Bradley (Florence Briggs Thayer)</Name><Sex>female</Sex><Age>38</Age><SibSp>1</SibSp><Parch>0</Parch><Ticket>PC 17599</Ticket><Fare>71.2833</Fare><Cabin>C85</Cabin><Embarked>C</Embarked></row>


Convert entire rows in the document.

In [7]:
for doc in docs[1:6]:  # skip header
    row = doc.page_content.split("\n")
    row_str = "<row>"
    for element in row:
        splitted_element = element.split(":")  # split by ":"
        value = splitted_element[-1]  # get value
        col = ":".join(splitted_element[:-1])  # get column name
        row_str += f"<{col}>{value.strip()}</{col}>"
    row_str += "</row>"
    print(row_str)

<row><PassengerId>2</PassengerId><Survived>1</Survived><Pclass>1</Pclass><Name>Cumings, Mrs. John Bradley (Florence Briggs Thayer)</Name><Sex>female</Sex><Age>38</Age><SibSp>1</SibSp><Parch>0</Parch><Ticket>PC 17599</Ticket><Fare>71.2833</Fare><Cabin>C85</Cabin><Embarked>C</Embarked></row>
<row><PassengerId>3</PassengerId><Survived>1</Survived><Pclass>3</Pclass><Name>Heikkinen, Miss. Laina</Name><Sex>female</Sex><Age>26</Age><SibSp>0</SibSp><Parch>0</Parch><Ticket>STON/O2. 3101282</Ticket><Fare>7.925</Fare><Cabin></Cabin><Embarked>S</Embarked></row>
<row><PassengerId>4</PassengerId><Survived>1</Survived><Pclass>1</Pclass><Name>Futrelle, Mrs. Jacques Heath (Lily May Peel)</Name><Sex>female</Sex><Age>35</Age><SibSp>1</SibSp><Parch>0</Parch><Ticket>113803</Ticket><Fare>53.1</Fare><Cabin>C123</Cabin><Embarked>S</Embarked></row>
<row><PassengerId>5</PassengerId><Survived>0</Survived><Pclass>3</Pclass><Name>Allen, Mr. William Henry</Name><Sex>male</Sex><Age>35</Age><SibSp>0</SibSp><Parch>0</

## UnstructuredCSVLoader 

`UnstructuredCSVLoader` có thể được sử dụng ở cả chế độ `single` và `elements`. Nếu bạn sử dụng trình tải ở chế độ "elements", tệp CSV sẽ là một phần tử Unstructured Table duy nhất. Nếu bạn sử dụng trình tải ở chế độ "elements", biểu diễn HTML của bảng sẽ có sẵn trong khóa `text_as_html` trong metadata của tài liệu.


In [9]:
from langchain_community.document_loaders.csv_loader import UnstructuredCSVLoader

# Generate UnstructuredCSVLoader instance with elements mode
loader = UnstructuredCSVLoader(file_path="data/titanic.csv", mode="elements")

docs = loader.load()

html_content = docs[0].metadata["text_as_html"]

# Partial output due to space constraints
print(html_content[:810]) 

<table><tr><td>PassengerId</td><td>Survived</td><td>Pclass</td><td>Name</td><td>Sex</td><td>Age</td><td>SibSp</td><td>Parch</td><td>Ticket</td><td>Fare</td><td>Cabin</td><td>Embarked</td></tr><tr><td>1</td><td>0</td><td>3</td><td>Braund, Mr. Owen Harris</td><td>male</td><td>22</td><td>1</td><td>0</td><td>A/5 21171</td><td>7.25</td><td/><td>S</td></tr><tr><td>2</td><td>1</td><td>1</td><td>Cumings, Mrs. John Bradley (Florence Briggs Thayer)</td><td>female</td><td>38</td><td>1</td><td>0</td><td>PC 17599</td><td>71.2833</td><td>C85</td><td>C</td></tr><tr><td>3</td><td>1</td><td>3</td><td>Heikkinen, Miss. Laina</td><td>female</td><td>26</td><td>0</td><td>0</td><td>STON/O2. 3101282</td><td>7.925</td><td/><td>S</td></tr><tr><td>4</td><td>1</td><td>1</td><td>Futrelle, Mrs. Jacques Heath (Lily May Peel)</td>


## DataFrameLoader

`Pandas` là một công cụ phân tích và thao tác dữ liệu mã nguồn mở cho ngôn ngữ lập trình Python. Thư viện này được sử dụng rộng rãi trong khoa học dữ liệu, máy học và nhiều lĩnh vực khác nhau để làm việc với dữ liệu.

`DataFrameLoader` của LangChain là một tiện ích mạnh mẽ được thiết kế để tích hợp liền mạch `DataFrames` của `Pandas` vào quy trình làm việc của LangChain.


In [10]:
import pandas as pd

df = pd.read_csv("./data/titanic.csv")
df.head(n=5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Parameters `page_content_column` (str) – Name of the column containing the page content. Defaults to “text”.

In [11]:
from langchain_community.document_loaders import DataFrameLoader

# The Name column of the DataFrame is specified to be used as the content of each document.
loader = DataFrameLoader(df, page_content_column="Name")

docs = loader.load()

print(docs[0].page_content)

Braund, Mr. Owen Harris


`Lazy Loading` for large tables. Avoid loading the entire table into memory

In [12]:
# Lazy load records from dataframe.
for row in loader.lazy_load():
    print(row)
    break  # print only the first row

page_content='Braund, Mr. Owen Harris' metadata={'PassengerId': 1, 'Survived': 0, 'Pclass': 3, 'Sex': 'male', 'Age': 22.0, 'SibSp': 1, 'Parch': 0, 'Ticket': 'A/5 21171', 'Fare': 7.25, 'Cabin': nan, 'Embarked': 'S'}
